## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer 
from pymystem3 import Mystem
from tqdm import tqdm
frame = pd.read_csv('/datasets/data.csv')

# Изучение данных


<h3>Посмотрим корректность названий столбцов</h3>

In [2]:
frame.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Все отлично, чтолбцы не содаржат пробелов, все знаки в одном регистре и на одном языке

<h3>Посмотрим общую информацию</h3>

In [3]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заметим, что присутствуют два столбца содержащих пропущенные значения. Типы столбцов соответсвуют информации хранящихся в них

Так же проверим величины хранящиеся в таблице

In [4]:
frame.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Сразу заметим, что в столбце days_employed and children and dob_years содержатся ненормальные для них значения

# Исправим невозможне значения в столбцах


<h4>Проверим, есть ли другие отрицательные значения(кроме "-1") в столбце отвечающего за количество детей</h4>

In [5]:
frame[frame['children'] < 0]['children'].unique()

array([-1])

Другие начения отсутсвуют. Скорее всего это техническая ошибка, которую следует исправить заменой. Более подробные размышления на эту тему будут приведены при обработке столбца days_employed

In [6]:
frame['children'].replace(-1,1,inplace = True)

Теперь у нас нет отрицательного количества детей

<h4>Теперь займемся столбцом days_employed, тут все намного интереснее</h4>

На 2020 пенсионный возраст женщин 55, мужчин 60. Заметим, что человек проиживший 100 лет и работающий с самого своего рождения может максимально иметь ~36500 дней рабочего стажа(это тоже сомнительно), в столбце ответственном за хранение стажа преимужественно отрицательные значения или значение превосходящие человеческие возможности. Приверим есть ли в данном столбце значения большие 0, но при этом не превосходящие человеческие возможности(36500 дней стажа) 

In [7]:
frame[(frame['days_employed'] >= 0) & (frame['days_employed'] < 36500)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Заметим, что таких объектов нет. Тут напрашивается два вывода, либо при выгрузке в документ данных произошла техническая ошибка из-за которой корректные данные стали отрицательными или при сборе информации были введены неверные значения. Отсюда можно прийти к трем решениям:
1. Откинуть минус и считать данные, которые являются отрицательными истинными, но содержашими ошибку артефакт в виде минуса. А значения не укладывающиеся в человеческие возможности привести к максимальному стажу (<b>я выбрал этот вариант</b>)  
<br>
1. Так как наша задача заключается в определении влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, то толбец days_employed можно исключить из выборки путем удаления  
<br>
1. Можно попробовать привести все некорректные значения столбца 'days_employed' к максимальному стажу опираясь на пол и возраст. Но это может силно испортить данные

Для получения максимального трудового стажа нам потребуется пол. Проверим все ли с ним впорядке

In [8]:
frame['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В поле gender мы можем увидеть 3 уникальных гендера ('F', 'M', 'XNA')

Отдельно рассмотрим что за гендр XNA

In [9]:
frame[frame['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Гендер 'XNA' встречается 1 раз в обекте с индексом 10701(исходной таблицы). С ним мы можем поступить двумя способами:

1. Не учитывать его вовсе и удалить из данных, он 1 из 21 тысячи  
1. Оставить его (<b>выибираю это</b>)

<h4>Напишем функцию, которая вычисляет максимальный рабочий стаж и в случае, если стаж больше возможного наменяет его на максимальный</h4>

Так же мы могли заметить, что у нас пресутствуют нулевые значения в возрасте, в 0 лет нас точно ни на какую работу не возьмут, поэтому заменим все 0 значения на 15 лет, так как минимальное после 0 в наших данных это 19 лет, мы всего лишь сдвинем минимальный возраст на более разумную гранцу

In [10]:
frame['dob_years'].replace(0,15,inplace = True)

Если сейчас мы запросим минимальный возраст, то мы сможем увидеть значение равное 15

In [11]:
frame['dob_years'].min()

15

In [12]:
def max_seniority(dataframe):
    '''Функция поиска максимального трудового стажа
    если стаж больше максимального, то
    функция возвращает максимально возможный стаж,
    основываясь на возраст человека и пол, в противном случает
    возвращает исходный трудовой стаж
    '''
    current_seniority = dataframe['days_employed']
    if current_seniority == np.nan:
        return current_seniority
    else:
        if dataframe['gender'] == 'M' and dataframe['dob_years'] > 60 : 
            max_years = 60
        elif dataframe['gender'] == 'F' and dataframe['dob_years'] > 55:
            max_years = 55
        else: 
            max_years = dataframe['dob_years']

        max_seniority = (max_years - 14) * 365
        if current_seniority > max_seniority:
            return max_seniority
        elif current_seniority < 0:
            return (current_seniority * -1)
        else:
            return current_seniority

In [13]:
frame['days_employed'] = frame.apply(max_seniority, axis = 1)

Проверим изменения

In [14]:
frame.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,4564.526196,43.363763,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,5236.358608,12.372407,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,15.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,923.819855,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2188.846985,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5505.044712,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Так гораздо лучше

### Вывод

1. Мы поняли, что в данных присутвовали артефакты
1. Таблица нуждается в очистке и исправления, которую частично мы смогли провести

### Предобработка данных

### Обработка пропусков

<h3>Возьмемся за устранение пропусков в стольбце days_employed</h3>

In [15]:
frame[frame['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Заметим, что там, где пропуск в days_employed, у того же обекта пропуск и в total_income. Пропусков 2174, что достаточно много для того что бы не обращать на них внимания и удалить их. Поэтому логично будет сгруппировать по полу и возрасту, а затем для каждой группы найти медиану и присвоить ее пропускам из той же группы

In [16]:
pivot_table_days_employed = frame.pivot_table(index=['dob_years'],
                                columns = 'gender',
                                values = 'days_employed',
                                aggfunc='median')

Теперь остается только перебрать пропуски в стаже работы и заполнить их в соответсвии со сводной таблицей. Напишем для этого специальную функцию

In [17]:
def fillna_in_days_employed(dataframe):
    """Функция возвращает медианное значение трудового стажа
    опираясь на пол и возраст, если натыкается на NaN. В противном случае
    возвращает переданное значение обратно
    """
    gender = dataframe['gender']
    age = dataframe['dob_years']
    if pd.isnull(dataframe['days_employed']):
        return pivot_table_days_employed.loc[age,gender]
    else:
        return dataframe['days_employed']

In [18]:
frame['days_employed'] = frame.apply(fillna_in_days_employed, axis = 1)

In [19]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в days_employed заполнены. Теперь займемся пропусками в total_income. Сложно сказать насколько доход в 2 млн нормален, поэтому, как мне кажется, тут необходимо заполнить пропуски медианным значением основываясь на возраст и пол

In [20]:
pivot_table_total_income = frame.pivot_table(index=['dob_years'],
                                    columns = 'gender',
                                    values = 'total_income',
                                    aggfunc='median')

In [21]:
def fillna_in_total_income(dataframe):
    """Функция возвращает медианное значение заработной платы
    опираясь на пол и возраст, если натыкается на NaN. В противном случае
    возвращает переданное значение обратно
    """
    gender = dataframe['gender']
    age = dataframe['dob_years']
    if pd.isnull(dataframe['total_income']):
        return pivot_table_total_income.loc[age,gender]
    else:
        return dataframe['total_income']

In [22]:
frame['total_income'] = frame.apply(fillna_in_total_income, axis = 1)

Пропуски в total_income заполнены. Пропусков в данных не осталось. Проверим это

In [23]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1. Любые данные не идеальны и могут содержать пропуски
1. Пропуски нужно заполнять основываясь на зависящие признаки, так данные получаются более точными

### Замена типа данных

In [24]:
frame['days_employed'] = frame.astype({'total_income':'int','days_employed':'int'})

### Вывод

1. Зачастую данные в таблице содержаться не в подходящем типе данных, но в данном случае в целом все верно
1. Для более высокой точности стоит привести целочисленные значения к типу int, а не оставлять во float

### Обработка дубликатов

Для начала приведем все текстовые значения к нижнему регистру, исключая гендер, он нам больше не понадобиться 

Проанализируем столбцы, в которых должны храниться только строковые значения

In [25]:
frame["education"].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Сразу можно заметить, что здесь присутсвуют дубликаты отличающиеся по регистру. Это нужно исправить

In [26]:
frame["education"] = frame["education"].str.lower()

In [27]:
frame["education"].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь все гораздо аккуратнее и без дубликатов. Сделаем тоже самое и для оставшихся столбцов

In [28]:
frame["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Тут все гораздо лучше, но все равно следуюет привести их к одному регистру

In [29]:
frame["family_status"] = frame["family_status"].str.lower()

In [30]:
frame["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Теперь перейдем к столбцу income_type

In [31]:
frame["income_type"].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Тут и без обработки все впорядке

Найдем количество дубликатов

In [32]:
frame.duplicated().sum()

74

Избавимся от них и востановим индексы

In [33]:
frame  = frame.drop_duplicates().reset_index(drop = True)

Проверим что у нас вышло

In [34]:
frame.duplicated().sum()

0

### Вывод

1. Были найдены дубликаты по регистру
2. Так же были найдены группы, отличавшиеся регистром букв в названии

### Лемматизация

In [35]:
m = Mystem()

In [36]:
dict_purpose = ['жилье','недвижимость','строительство',"автомобиль",'образование',"свадьба","ремонт"]

In [37]:
dict_purpose_core = [m.lemmatize(word) for word in dict_purpose]
dict_purpose_core = [element[0] for element in dict_purpose_core]

In [38]:
dict_purpose_core

['жилье',
 'недвижимость',
 'строительство',
 'автомобиль',
 'образование',
 'свадьба',
 'ремонт']

In [39]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21451 entries, 0 to 21450
Data columns (total 12 columns):
children            21451 non-null int64
days_employed       21451 non-null object
dob_years           21451 non-null int64
education           21451 non-null object
education_id        21451 non-null int64
family_status       21451 non-null object
family_status_id    21451 non-null int64
gender              21451 non-null object
income_type         21451 non-null object
debt                21451 non-null int64
total_income        21451 non-null float64
purpose             21451 non-null object
dtypes: float64(1), int64(5), object(6)
memory usage: 2.0+ MB


In [40]:
def find_core(text):
    for lemma in m.lemmatize(text):
        if lemma in dict_purpose_core:
            return lemma
                


In [41]:
frame["purpose_type"] = frame["purpose"].apply(find_core)

In [42]:
frame['purpose_type'].unique()

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость',
       'строительство', 'ремонт'], dtype=object)

In [43]:
frame['purpose_type'].replace("жилье",'недвижимость',inplace = True)
frame['purpose_type'].replace("ремонт",'строительство',inplace = True)

Жилье и недвижимость это грубо говоря одно и тоже, а ремонт и строительство это достаточно близкие понятие, если их сложить, то рапределения групп между имеющих задолжность и не имеющих измениться на 0.03, чем можно пренебречь

### Вывод

Мы видели, что в столбце purpose находились дублирующие причины, которые отличались формулировкой, поэтому причины стоило лемматизировать. Теперь стоит еще раз проверить дубликаты 

In [44]:
frame.duplicated().sum()

0

Заметим, что у нас появились еще 255 дубликатов, удалим и их

In [45]:
frame  = frame.drop_duplicates().reset_index(drop = True)

Проверим

In [46]:
frame.duplicated().sum()

0

Все отлично, теперь перейдем к категоризации данных

### Категоризация данных

Категоризируем количесвто детей разделим и на две группы "семьи без детей" и "многодетные". Напишем для этого функцию

In [47]:
def count_of_child(row):
    if row == 0:
        return 'семьи без детей'
    else:
        return 'многодетные'

In [48]:
frame['type_of_family'] = frame['children'].apply(count_of_child)

Теперь поделин на категории заработные платы. 
Заметим, что заработная плату можно разделить на три категории:

низкая 20 000 - 100 000

средняя 100 000 - 200 000

высокая болше 200 000

Напишем функцию для категоризации заработных плат

In [49]:
def classifier_total_income(row):
    if row <= 100000:
        return 'низкая'
    elif 100000 < row <= 200000:
        return "средняя"
    else:
        return "высокая"

In [50]:
frame['total_income_status'] = frame['total_income'].apply(classifier_total_income)

### Мысли по поводу определения уровней дохода:

Рассмотрим распрелеоение значений в столбце 'total_income'

In [51]:
frame['total_income'].describe().astype('int')

count      21451
mean      165236
std        98218
min        20667
25%       107305
50%       143481
75%       195820
max      2265604
Name: total_income, dtype: int64

Заметим, что 50% заработных плат меньше 143481, а 25% ниже 107305. Поэтому я посчитал, что наиболее логично в данном случае будет выделить определения основываясь на квартили 
1. 0% - 25%: низкая
1. 25% - 75%: средняя
1. 75% - 100%: высокая

### Вывод

Теперь основываясь на категории мы сможем изучить влияет ли какая то категория на вероятность задолжности по кредиту 

### Проверим гипотезы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Да, есть теперь выразим это в цифрах

решение 1, через группировку

In [52]:
frame_of_family = frame.groupby(['type_of_family','debt']).agg({'debt':["count"]})

In [53]:
frame_of_family

debt
                      count
type_of_family  debt       
многодетные     0      6685
                1       678
семьи без детей 0     13025
                1      1063

In [54]:

def fraction(table, max_index,row_names):
    count = 0
    for i in range(1,max_index + 1, 2):
        total = table.iloc[i] + table.iloc[i-1]
        fraction_withchild_TrueDebt  = (table.iloc[i] / total)[0]
        print("Вероятность клиента принадлежащего к группе \"{}\" стать должником составляет {:.1%}".format(row_names[count],fraction_withchild_TrueDebt))
        count += 1

In [55]:
fraction(frame_of_family,3,['многодетные',"семьи без детей"])

Вероятность клиента принадлежащего к группе "многодетные" стать должником составляет 9.2%
Вероятность клиента принадлежащего к группе "семьи без детей" стать должником составляет 7.5%


### Вывод

Как и говорилось выше, зависимость есть. Как мы смогли выяснить, многодетные семьи на 1.7% чаще имеют задолжность по кредиту  

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Да, есть теперь выразим это в цифрах

In [56]:
frame_of_family_status = frame.groupby(['family_status','debt']).agg({'debt':["count"]})

In [57]:
frame_of_family_status

debt
                            count
family_status         debt       
в разводе             0      1110
                      1        85
вдовец / вдова        0       896
                      1        63
гражданский брак      0      3763
                      1       388
женат / замужем       0     11405
                      1       931
не женат / не замужем 0      2536
                      1       274

In [58]:
fraction(frame_of_family_status,9,
         ['в разводе',"вдовец / вдова","гражданский брак","женат / замужем",'не женат / не замужем'])

Вероятность клиента принадлежащего к группе "в разводе" стать должником составляет 7.1%
Вероятность клиента принадлежащего к группе "вдовец / вдова" стать должником составляет 6.6%
Вероятность клиента принадлежащего к группе "гражданский брак" стать должником составляет 9.3%
Вероятность клиента принадлежащего к группе "женат / замужем" стать должником составляет 7.5%
Вероятность клиента принадлежащего к группе "не женат / не замужем" стать должником составляет 9.8%


### Вывод

Не состоящие в браке наиболее склонны к задержке выплат, в то же время вдовецы/вдвы наименьше всех склонны к задолжностям по кредиту 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [59]:
frame_of_income_status = frame.groupby(['total_income_status','debt']).agg({'debt':["count"]})

In [60]:
frame_of_income_status

debt
                          count
total_income_status debt       
высокая             0      4708
                    1       358
низкая              0      4110
                    1       354
средняя             0     10892
                    1      1029

In [61]:
fraction(frame_of_income_status,5,['высокая',"низкая","средняя"])

Вероятность клиента принадлежащего к группе "высокая" стать должником составляет 7.1%
Вероятность клиента принадлежащего к группе "низкая" стать должником составляет 7.9%
Вероятность клиента принадлежащего к группе "средняя" стать должником составляет 8.6%


### Вывод

Наибольшая вероятность 8.6% стать должником есть у людей, получающих среднюю зарплату

- Как разные цели кредита влияют на его возврат в срок?

In [62]:
frame_of_purpose_type = frame.groupby(['purpose_type','debt']).agg({'debt':["count"]})


In [63]:
frame_of_purpose_type

debt
                   count
purpose_type  debt      
автомобиль    0     3902
              1      403
недвижимость  0     7721
              1      603
образование   0     3643
              1      370
свадьба       0     2138
              1      186
строительство 0     2306
              1      179

In [64]:
fraction(frame_of_purpose_type,9,
        ['автомобиль',"недвижимость","образование","свадьба",'строительство'])

Вероятность клиента принадлежащего к группе "автомобиль" стать должником составляет 9.4%
Вероятность клиента принадлежащего к группе "недвижимость" стать должником составляет 7.2%
Вероятность клиента принадлежащего к группе "образование" стать должником составляет 9.2%
Вероятность клиента принадлежащего к группе "свадьба" стать должником составляет 8.0%
Вероятность клиента принадлежащего к группе "строительство" стать должником составляет 7.2%


### Вывод

Наибольшая вероятность 9.4% стать должником есть у людей, получающих кредитные средста на операции с автомобилем

### Общий вывод

Основываясь на полученные данные можно прийти к следующему выводу:
Семейное положение и наличие детей в семье безусловно влияют на факт погашения крудита.
Так мы можем увидеть, что вероятность клиента принадлежащего к группе "многодетные" стать должником составляет 9.2%, а вероятность клиента принадлежащего к группе "не женат / не замужем" стать должником составляет 9.8%. Поэтому логично будет предположить, что наиболее безопасными для банка заемщиками являются семьи без детей и люди являющиеся вдовой/вдовцом